<img src="../../img/elevation.png" alt="" style="width: 100%; border-radius: 20px;"/>

## Objective

## Scope
In this notebook, we aim to generate the altitude values that are inherently only given for swiss dataset. To ensure consistency, we will calculate them for Germany and also for Switzerland. Für diesen Zweck haben wir mehrere Ansätze und Datenquellen angezapft und dabei zwei Ansätze aufgrund von zu langsamer Berechnungszeit und issues mit falschen Altitudes verworfen. Letztendlich verwenden wir Earth Engine von Google, um schnelle und korrekte Altitude-Daten zu generieren. Am Ende wird es eine Ersteinschätzung geben, die einschätzt, inwiefern das Hinzufügen eines altitude Wertes die Beurteilung der Validität eines Datenpunktes erlaubt. Hierzu wird beispielhaft der Bergpieper und der Seeadler herangezogen. 

Am Ende soll ein PCA

In [1]:
import pandas as pd
import sys
sys.path.append('../')
from utils.copernicus import CopernicusDEM
import requests

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', False)

In [8]:
df_path = r'D:\Simon\Documents\GP\data\master_train.csv'
df = pd.read_csv(df_path, index_col=0)

C:\Users\simon\AppData\Local\Temp\ipykernel_12984\3317260435.py:2: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(df_path, index_col=0)


In [9]:
df.head(5)

,id_sighting,id_species,name_species,date,coord_lat,coord_lon,altitude,total_count,atlas_code,country,eea_grid_id,urban_area_percent,industrial_area_percent,agriculture_area_percent,forest_area_percent,grassland_area_percent,shrubland_area_percent,coastal_area_percent,rocky_area_percent,sparsley_vegetated_area_percent,burnt_area_percent,glacier_area_percent,wetlands_area_percent,water_area_percent,land_use_coord,land_use_coord_numeric,unclassified_area_percent
0,29666972,8,Haubentaucher,2018-01-01,53.157760,8.676993,-1,0,NaN,de,50kmE4200N3300,0.191298,0.000000,0.399560,0.000000,0.0,0.021766,0.0,0.0,0.0,0.0,0.0,0.0,0.387377,water,14,0.0
1,29654244,397,Schwarzkehlchen,2018-01-01,53.127639,8.957263,1,2,NaN,de,50kmE4250N3300,0.000000,0.000000,1.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,agriculture,3,0.0
2,29654521,463,Wiesenpieper,2018-01-01,50.850941,12.146953,269,2,NaN,de,50kmE4450N3050,0.405476,0.000000,0.594524,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,urban,1,0.0
3,29666414,8,Haubentaucher,2018-01-01,51.076006,11.038316,157,8,NaN,de,50kmE4350N3100,0.000000,0.004445,0.376111,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.619445,water,14,0.0
4,29656211,8,Haubentaucher,2018-01-01,51.389380,7.067282,52,10,NaN,de,50kmE4100N3100,0.362214,0.000000,0.024020,0.211704,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.402062,urban,1,0.0


In [10]:
print(df.altitude.value_counts())

altitude
 0       115450
 430     88178 
-1       51270 
-2       43926 
 400     38170 
         ...   
 2893    1     
 3262    1     
 3124    1     
 3001    1     
 2971    1     
Name: count, Length: 2899, dtype: int64


In [11]:
df.drop(columns=['altitude'], inplace=True)

### Add elevation features

In [6]:
copernicus = CopernicusDEM(raster_paths=['D:\\Simon\\Documents\\GP\\data\\util_files\\EU_DEM\\eu_dem_v11_E40N20.TIF', 'D:\\Simon\\Documents\\GP\\data\\util_files\\EU_DEM\\eu_dem_v11_E40N30.TIF'])

In [7]:
df = copernicus.get_elevation(df, lat_col='coord_lat', lon_col='coord_lon')

In [8]:
df['altitude'] = df['elevation']
df.drop(columns=['elevation'], inplace=True)

In [ ]:
df.head(2)

### Check for high negative altitude values

In [ ]:
print(df.altitude.value_counts())

In [ ]:
min_altitude = df.altitude.min()
df_min_altitude = df[df.altitude == min_altitude]
df_min_altitude.head(5)

### Set 3.402823e+38 altitude to 0

In [ ]:
df.loc[df['altitude'] == df.altitude.min(), 'altitude'] = 0

In [ ]:
print(df.altitude.value_counts())

### Earth Engine Elevation 

In [ ]:
import ee
import pandas as pd

# Initialize the Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-simonbirk')


# Assuming df is your DataFrame and it has columns 'coord_lat' and 'coord_lon'
def get_elevation(lat, lon):
    # Create a point with the coordinates
    point = ee.Geometry.Point([lon, lat])

    # Get the elevation data
    dataset = ee.Image('USGS/SRTMGL1_003')

    # Sample the point and get the elevation
    elevation = dataset.sample(point, 30).first().get('elevation').getInfo()
    return elevation

# Apply the function to each row in the DataFrame
df['altitude'] = df.apply(lambda row: get_elevation(row['coord_lat'], row['coord_lon']), axis=1)


In [17]:
df.head(5)

,id_sighting,id_species,name_species,date,coord_lat,coord_lon,total_count,atlas_code,country,eea_grid_id,urban_area_percent,industrial_area_percent,agriculture_area_percent,forest_area_percent,grassland_area_percent,shrubland_area_percent,coastal_area_percent,rocky_area_percent,sparsley_vegetated_area_percent,burnt_area_percent,glacier_area_percent,wetlands_area_percent,water_area_percent,land_use_coord,land_use_coord_numeric,unclassified_area_percent,altitude
0,29666972,8,Haubentaucher,2018-01-01,53.157760,8.676993,0,NaN,de,50kmE4200N3300,0.191298,0.000000,0.399560,0.000000,0.0,0.021766,0.0,0.0,0.0,0.0,0.0,0.0,0.387377,water,14,0.0,-1.0
1,29654244,397,Schwarzkehlchen,2018-01-01,53.127639,8.957263,2,NaN,de,50kmE4250N3300,0.000000,0.000000,1.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,agriculture,3,0.0,0.0
2,29654521,463,Wiesenpieper,2018-01-01,50.850941,12.146953,2,NaN,de,50kmE4450N3050,0.405476,0.000000,0.594524,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,urban,1,0.0,269.0
3,29666414,8,Haubentaucher,2018-01-01,51.076006,11.038316,8,NaN,de,50kmE4350N3100,0.000000,0.004445,0.376111,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.619445,water,14,0.0,157.0
4,29656211,8,Haubentaucher,2018-01-01,51.389380,7.067282,10,NaN,de,50kmE4100N3100,0.362214,0.000000,0.024020,0.211704,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.402062,urban,1,0.0,51.0


### Experiment with elevation APIs

In [12]:
def get_elevation(lat, long):
    query = f'https://api.open-elevation.com/api/v1/lookup?locations={lat},{long}'
    r = requests.get(query).json()
    print(r)
    elevation = r['results'][0]['elevation']
    return elevation

In [13]:
def get_elevation2(lat, long):
    query = f'https://api.opentopodata.org/v1/eudem25m?locations={lat},{long}'
    r = requests.get(query).json()
    elevation = r['results'][0]['elevation']
    return elevation

In [14]:
df_min_altitude['altitude'] = df_min_altitude.head(10).apply(lambda row: get_elevation(row['coord_lat'], row['coord_lon']), axis=1)

{'results': [{'latitude': 53.780461, 'longitude': 7.947729, 'elevation': 0.0}]}
{'results': [{'latitude': 53.702563, 'longitude': 7.740776, 'elevation': 0.0}]}
{'results': [{'latitude': 54.50565, 'longitude': 9.553208, 'elevation': 0.0}]}
{'results': [{'latitude': 54.758682, 'longitude': 8.295368, 'elevation': 4.0}]}
{'results': [{'latitude': 54.758271, 'longitude': 8.294853, 'elevation': 1.0}]}
{'results': [{'latitude': 54.34975, 'longitude': 10.624524, 'elevation': -3.0}]}
{'results': [{'latitude': 54.34975, 'longitude': 10.624524, 'elevation': -3.0}]}
{'results': [{'latitude': 54.34975, 'longitude': 10.624524, 'elevation': -3.0}]}
{'results': [{'latitude': 54.266281, 'longitude': 13.313914, 'elevation': 0.0}]}
{'results': [{'latitude': 54.34531, 'longitude': 13.581008, 'elevation': 0.0}]}


C:\Users\simon\AppData\Local\Temp\ipykernel_17476\2604693980.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_min_altitude['altitude'] = df_min_altitude.head(10).apply(lambda row: get_elevation(row['coord_lat'], row['coord_lon']), axis=1)


In [15]:
df_min_altitude.head(5)

,id_sighting,id_species,name_species,date,timing,coord_lat,coord_lon,precision,altitude,total_count,atlas_code,id_observer,country,eea_grid_id
56,29660454,8.0,Haubentaucher,2018-01-01,NaN,53.780461,7.947729,precise,0.0,1.0,NaN,77774.0,de,50kmE4150N3400
131,29652796,158.0,Seeadler,2018-01-01,NaN,53.702563,7.740776,square,0.0,1.0,NaN,43003.0,de,50kmE4150N3400
278,29667163,8.0,Haubentaucher,2018-01-01,NaN,54.505650,9.553208,precise,0.0,2.0,NaN,37768.0,de,50kmE4250N3450
301,29655667,8.0,Haubentaucher,2018-01-01,02:44,54.758682,8.295368,precise,4.0,1.0,NaN,42717.0,de,50kmE4200N3500
302,29653528,8.0,Haubentaucher,2018-01-01,03:14,54.758271,8.294853,precise,1.0,2.0,NaN,45902.0,de,50kmE4200N3500


### Merge new Altitude data into old data

In [16]:
df_path = 'D:\\Simon\\Documents\\GP\\data\\datasets\\selected_bird_species_with_grids_50km.csv'
df = pd.read_csv(df_path, index_col=0)
df.reset_index(inplace=True, drop=True)

C:\Users\simon\AppData\Local\Temp\ipykernel_17476\247923559.py:2: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(df_path, index_col=0)


In [17]:
df_path_alt = 'D:\\Simon\\Documents\\GP\\data\\datasets\\altitude\\dbird_export_altitude_202312140925.csv'
df_alt = pd.read_csv(df_path_alt, sep=';')

In [ ]:
df_alt.head(3)

In [18]:
df_de = df[df['country'] == 'de']

merged_df_de = df_de.merge(df_alt, left_on='id_sighting', right_on='ornitho_id', how='left')
merged_df_de.drop(columns=['ornitho_id', 'altitude_x'], inplace=True)
merged_df_de.rename(columns={'altitude_y': 'altitude'}, inplace=True)

df.update(merged_df_de)

In [19]:
df.head(3)

,id_sighting,id_species,name_species,date,timing,coord_lat,coord_lon,precision,altitude,total_count,atlas_code,id_observer,country,eea_grid_id
0,29666972,8.0,Haubentaucher,2018-01-01,NaN,53.157760,8.676993,place,-1,0.0,NaN,37718.0,de,50kmE4200N3300
1,29654244,397.0,Schwarzkehlchen,2018-01-01,NaN,53.127639,8.957263,square,1,2.0,NaN,37803.0,de,50kmE4250N3300
2,29654521,463.0,Wiesenpieper,2018-01-01,NaN,50.850941,12.146953,place,269,2.0,NaN,39627.0,de,50kmE4450N3050


In [20]:
print(df.altitude.value_counts())

altitude
 0       115450
 430     88178 
-1       51270 
-2       43926 
 400     38170 
         ...   
 2893    1     
 3262    1     
 3124    1     
 3001    1     
 2971    1     
Name: count, Length: 2899, dtype: int64


In [21]:
min_altitude = df.altitude.min()
df_min_altitude = df[df.altitude == min_altitude]
df_min_altitude.head(5)

,id_sighting,id_species,name_species,date,timing,coord_lat,coord_lon,precision,altitude,total_count,atlas_code,id_observer,country,eea_grid_id
172472,57776355,463.0,Wiesenpieper,2021-05-03,12:08,50.958478,6.674523,precise,-34,1.0,NaN,84909.0,de,50kmE4050N3050
172473,57790281,463.0,Wiesenpieper,2021-05-03,12:08,50.958478,6.674523,precise,-34,1.0,NaN,88575.0,de,50kmE4050N3050


In [23]:
df_path = 'D:\\Simon\\Documents\\GP\\data\\datasets\\selected_bird_species_with_grids_50km.csv'
# df_path = 'D:\\Simon\\Documents\\GP\\data\\datasets\\selected_bird_species_50km_elevation.csv'
df.to_csv(df_path)